fastNLP10 分钟上手教程
-------

fastNLP提供方便的数据预处理，训练和测试模型的功能

如果您还没有通过pip安装fastNLP，可以执行下面的操作加载当前模块

In [4]:
import sys
sys.path.append("../")

DataSet & Instance
------

fastNLP用DataSet和Instance保存和处理数据。每个DataSet表示一个数据集，每个Instance表示一个数据样本。一个DataSet存有多个Instance，每个Instance可以自定义存哪些内容。

有一些read_*方法，可以轻松从文件读取数据，存成DataSet。

In [1]:
from fastNLP import DataSet
from fastNLP import Instance

# 从csv读取数据到DataSet
dataset = DataSet.read_csv('sample_data/tutorial_sample_dataset.csv', headers=('raw_sentence', 'label'), sep='\t')
print(len(dataset))

77


In [2]:
# 使用数字索引[k]，获取第k个样本
print(dataset[0])

# 索引也可以是负数
print(dataset[-3])

{'raw_sentence': A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story . type=str,
'label': 1 type=str}
{'raw_sentence': The plot is romantic comedy boilerplate from start to finish . type=str,
'label': 2 type=str}


## Instance
Instance表示一个样本，由一个或多个field（域，属性，特征）组成，每个field有名字和值。

在初始化Instance时即可定义它包含的域，使用 "field_name=field_value"的写法。

In [3]:
# DataSet.append(Instance)加入新数据
dataset.append(Instance(raw_sentence='fake data', label='0'))
dataset[-1]

{'raw_sentence': fake data type=str,
'label': 0 type=str}

## DataSet.apply方法
数据预处理利器

In [4]:
# 将所有数字转为小写
dataset.apply(lambda x: x['raw_sentence'].lower(), new_field_name='raw_sentence')
print(dataset[0])

{'raw_sentence': a series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story . type=str,
'label': 1 type=str}


In [5]:
# label转int
dataset.apply(lambda x: int(x['label']), new_field_name='label')
print(dataset[0])

{'raw_sentence': a series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story . type=str,
'label': 1 type=int}


In [6]:
# 使用空格分割句子
def split_sent(ins):
    return ins['raw_sentence'].split()
dataset.apply(split_sent, new_field_name='words')
print(dataset[0])

{'raw_sentence': a series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story . type=str,
'label': 1 type=int,
'words': ['a', 'series', 'of', 'escapades', 'demonstrating', 'the', 'adage', 'that', 'what', 'is', 'good', 'for', 'the', 'goose', 'is', 'also', 'good', 'for', 'the', 'gander', ',', 'some', 'of', 'which', 'occasionally', 'amuses', 'but', 'none', 'of', 'which', 'amounts', 'to', 'much', 'of', 'a', 'story', '.'] type=list}


In [7]:
# 增加长度信息
dataset.apply(lambda x: len(x['words']), new_field_name='seq_len')
print(dataset[0])

{'raw_sentence': a series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story . type=str,
'label': 1 type=int,
'words': ['a', 'series', 'of', 'escapades', 'demonstrating', 'the', 'adage', 'that', 'what', 'is', 'good', 'for', 'the', 'goose', 'is', 'also', 'good', 'for', 'the', 'gander', ',', 'some', 'of', 'which', 'occasionally', 'amuses', 'but', 'none', 'of', 'which', 'amounts', 'to', 'much', 'of', 'a', 'story', '.'] type=list,
'seq_len': 37 type=int}


## DataSet.drop
筛选数据

In [8]:
# 删除低于某个长度的词语
dataset.drop(lambda x: x['seq_len'] <= 3)
print(len(dataset))

77


## 配置DataSet
1. 哪些域是特征，哪些域是标签
2. 切分训练集/验证集

In [9]:
# 设置DataSet中，哪些field要转为tensor

# set target，loss或evaluate中的golden，计算loss，模型评估时使用
dataset.set_target("label")
# set input，模型forward时使用
dataset.set_input("words")

In [10]:
# 分出测试集、训练集

test_data, train_data = dataset.split(0.3)
print(len(test_data))
print(len(train_data))

54
23


Vocabulary
------

fastNLP中的Vocabulary轻松构建词表，将词转成数字

In [11]:
from fastNLP import Vocabulary

# 构建词表, Vocabulary.add(word)
vocab = Vocabulary(min_freq=2)
train_data.apply(lambda x: [vocab.add(word) for word in x['words']])
vocab.build_vocab()

# index句子, Vocabulary.to_index(word)
train_data.apply(lambda x: [vocab.to_index(word) for word in x['words']], new_field_name='words')
test_data.apply(lambda x: [vocab.to_index(word) for word in x['words']], new_field_name='words')


print(test_data[0])

{'raw_sentence': the performances are an absolute joy . type=str,
'label': 4 type=int,
'words': [3, 1, 1, 26, 1, 1, 2] type=list,
'seq_len': 7 type=int}


In [12]:
# 如果你们需要做强化学习或者GAN之类的项目，你们也可以使用这些数据预处理的工具
from fastNLP.core.batch import Batch
from fastNLP.core.sampler import RandomSampler

batch_iterator = Batch(dataset=train_data, batch_size=2, sampler=RandomSampler())
for batch_x, batch_y in batch_iterator:
    print("batch_x has: ", batch_x)
    print("batch_y has: ", batch_y)
    break

batch_x has:  {'words': tensor([[ 15,  72,  15,  73,  74,   7,   3,  75,   6,   3,  16,  16,
          76,   2],
        [ 15,  72,  15,  73,  74,   7,   3,  75,   6,   3,  16,  16,
          76,   2]])}
batch_y has:  {'label': tensor([ 1,  1])}


# Model
定义一个PyTorch模型

In [15]:
from fastNLP.models import CNNText
model = CNNText((len(vocab), 50), num_classes=5, padding=2, dropout=0.1)
model

CNNText(
  (embed): Embedding(
    77, 50
    (dropout): Dropout(p=0.0)
  )
  (conv_pool): ConvMaxpool(
    (convs): ModuleList(
      (0): Conv1d(50, 3, kernel_size=(3,), stride=(1,), padding=(2,))
      (1): Conv1d(50, 4, kernel_size=(4,), stride=(1,), padding=(2,))
      (2): Conv1d(50, 5, kernel_size=(5,), stride=(1,), padding=(2,))
    )
  )
  (dropout): Dropout(p=0.1)
  (fc): Linear(
    (linear): Linear(in_features=12, out_features=5, bias=True)
  )
)

这是上述模型的forward方法。如果你不知道什么是forward方法，请参考我们的PyTorch教程。

注意两点：
1. forward参数名字叫**word_seq**，请记住。
2. forward的返回值是一个**dict**，其中有个key的名字叫**output**。

```Python
    def forward(self, word_seq):
        """

        :param word_seq: torch.LongTensor, [batch_size, seq_len]
        :return output: dict of torch.LongTensor, [batch_size, num_classes]
        """
        x = self.embed(word_seq)  # [N,L] -> [N,L,C]
        x = self.conv_pool(x)  # [N,L,C] -> [N,C]
        x = self.dropout(x)
        x = self.fc(x)  # [N,C] -> [N, N_class]
        return {'output': x}
```

这是上述模型的predict方法，是用来直接输出该任务的预测结果，与forward目的不同。

注意两点：
1. predict参数名也叫**word_seq**。
2. predict的返回值是也一个**dict**，其中有个key的名字叫**predict**。

```
    def predict(self, word_seq):
        """

        :param word_seq: torch.LongTensor, [batch_size, seq_len]
        :return predict: dict of torch.LongTensor, [batch_size, seq_len]
        """
        output = self(word_seq)
        _, predict = output['output'].max(dim=1)
        return {'predict': predict}
```

Trainer & Tester
------

使用fastNLP的Trainer训练模型

In [16]:
from fastNLP import Trainer
from copy import deepcopy
from fastNLP.core.losses import CrossEntropyLoss
from fastNLP.core.metrics import AccuracyMetric


# 更改DataSet中对应field的名称，与模型的forward的参数名一致
# 因为forward的参数叫word_seq, 所以要把原本叫words的field改名为word_seq
# 这里的演示是让你了解这种**命名规则**
train_data.rename_field('words', 'word_seq')
test_data.rename_field('words', 'word_seq')

# 顺便把label换名为label_seq
train_data.rename_field('label', 'label_seq')
test_data.rename_field('label', 'label_seq')

### loss
训练模型需要提供一个损失函数

下面提供了一个在分类问题中常用的交叉熵损失。注意它的**初始化参数**。

pred参数对应的是模型的forward返回的dict的一个key的名字，这里是"output"。

target参数对应的是dataset作为标签的field的名字，这里是"label_seq"。

In [17]:
loss = CrossEntropyLoss(pred="output", target="label_seq")

### Metric
定义评价指标

这里使用准确率。参数的“命名规则”跟上面类似。

pred参数对应的是模型的predict方法返回的dict的一个key的名字，这里是"predict"。

target参数对应的是dataset作为标签的field的名字，这里是"label_seq"。

In [18]:
metric = AccuracyMetric(pred="predict", target="label_seq")

In [19]:
# 实例化Trainer，传入模型和数据，进行训练
# 先在test_data拟合（确保模型的实现是正确的）
copy_model = deepcopy(model)
overfit_trainer = Trainer(model=copy_model, train_data=test_data, dev_data=test_data,
                          loss=loss,
                          metrics=metric,
                          save_path=None,
                          batch_size=32,
                          n_epochs=5)
overfit_trainer.train()

input fields after batch(if batch size is 2):
	word_seq: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 11]) 
target fields after batch(if batch size is 2):
	label_seq: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 



NameError: 
Problems occurred when calling CNNText.forward(self, words, seq_len=None)
	missing param: ['words']
	unused field: ['word_seq']
	Suggestion: You need to provide ['words'] in DataSet and set it as input. 

In [22]:
# 用train_data训练，在test_data验证
trainer = Trainer(model=model, train_data=train_data, dev_data=test_data,
                  loss=CrossEntropyLoss(pred="output", target="label_seq"),
                  metrics=AccuracyMetric(pred="predict", target="label_seq"),
                  save_path=None,
                  batch_size=32,
                  n_epochs=5)
trainer.train()
print('Train finished!')

input fields after batch(if batch size is 2):
	word_seq: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 20]) 
target fields after batch(if batch size is 2):
	label_seq: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 

training epochs started 2019-01-12 17-09-05


Evaluation at Epoch 1/5. Step:1/5. AccuracyMetric: acc=0.37037
Evaluation at Epoch 2/5. Step:2/5. AccuracyMetric: acc=0.37037
Evaluation at Epoch 3/5. Step:3/5. AccuracyMetric: acc=0.462963
Evaluation at Epoch 4/5. Step:4/5. AccuracyMetric: acc=0.425926
Evaluation at Epoch 5/5. Step:5/5. AccuracyMetric: acc=0.481481

In Epoch:5/Step:5, got best dev performance:AccuracyMetric: acc=0.481481
Reloaded the best model.
Train finished!


In [23]:
# 调用Tester在test_data上评价效果
from fastNLP import Tester

tester = Tester(data=test_data, model=model, metrics=AccuracyMetric(pred="predict", target="label_seq"),
                batch_size=4)
acc = tester.test()
print(acc)

[tester] 
AccuracyMetric: acc=0.481481
{'AccuracyMetric': {'acc': 0.481481}}


# In summary

## fastNLP Trainer的伪代码逻辑
### 1. 准备DataSet，假设DataSet中共有如下的fields
    ['raw_sentence', 'word_seq1', 'word_seq2', 'raw_label','label']
    通过
        DataSet.set_input('word_seq1', word_seq2', flag=True)将'word_seq1', 'word_seq2'设置为input
    通过
        DataSet.set_target('label', flag=True)将'label'设置为target
### 2. 初始化模型
    class Model(nn.Module):
        def __init__(self):
            xxx
        def forward(self, word_seq1, word_seq2):
            # (1) 这里使用的形参名必须和DataSet中的input field的名称对应。因为我们是通过形参名, 进行赋值的
            # (2) input field的数量可以多于这里的形参数量。但是不能少于。
            xxxx
            # 输出必须是一个dict
### 3. Trainer的训练过程
    (1) 从DataSet中按照batch_size取出一个batch，调用Model.forward
    (2) 将 Model.forward的结果 与 标记为target的field 传入Losser当中。
           由于每个人写的Model.forward的output的dict可能key并不一样，比如有人是{'pred':xxx}, {'output': xxx}; 
           另外每个人将target可能也会设置为不同的名称, 比如有人是label, 有人设置为target；
        为了解决以上的问题，我们的loss提供映射机制
           比如CrossEntropyLosser的需要的输入是(prediction, target)。但是forward的output是{'output': xxx}; 'label'是target
           那么初始化losser的时候写为CrossEntropyLosser(prediction='output', target='label')即可
     (3) 对于Metric是同理的
         Metric计算也是从 forward的结果中取值 与 设置target的field中取值。 也是可以通过映射找到对应的值        
 
       

## 一些问题.
### 1. DataSet中为什么需要设置input和target
    只有被设置为input或者target的数据才会在train的过程中被取出来
    (1.1) 我们只会在设置为input的field中寻找传递给Model.forward的参数。
    (1.2) 我们在传递值给losser或者metric的时候会使用来自: 
            (a)Model.forward的output
            (b)被设置为target的field
          

### 2. 我们是通过forwad中的形参名将DataSet中的field赋值给对应的参数
     (1.1) 构建模型过程中，
      例如:
          DataSet中x，seq_lens是input，那么forward就应该是
          def forward(self, x, seq_lens):
              pass
          我们是通过形参名称进行匹配的field的
       


### 1. 加载数据到DataSet
### 2. 使用apply操作对DataSet进行预处理
      (2.1) 处理过程中将某些field设置为input，某些field设置为target
### 3. 构建模型
      (3.1) 构建模型过程中，需要注意forward函数的形参名需要和DataSet中设置为input的field名称是一致的。
      例如:
          DataSet中x，seq_lens是input，那么forward就应该是
          def forward(self, x, seq_lens):
              pass
          我们是通过形参名称进行匹配的field的
      (3.2) 模型的forward的output需要是dict类型的。
          建议将输出设置为{"pred": xx}.
          
